# Module 6: CrewAI

## Applied AI Scientist Field Notes - Expanded Edition

---


## Module 6: CrewAI - Role-Based Task Orchestration

### Topics
1. Agent roles and expertise
2. Task delegation
3. Sequential vs hierarchical processes
4. Memory management
5. Tool integration
6. Output validation

---

In [ ]:
%pip install -q crewai crewai-tools

print('CrewAI installed!')

### Section 1: Role-Based Agents

CrewAI concepts:
- **Agent**: Has role, goal, backstory, tools
- **Task**: Work unit with expected output
- **Crew**: Orchestrates agents and tasks
- **Process**: Sequential or hierarchical
- **Memory**: Shared context across agents

In [ ]:
class CrewAgent:
    def __init__(self, role: str, goal: str, backstory: str, tools: list = None):
        self.role = role
        self.goal = goal
        self.backstory = backstory
        self.tools = tools or []
    
    def execute_task(self, task: str) -> str:
        return f'[{self.role}] completed: {task[:50]}...'

class Task:
    def __init__(self, description: str, expected_output: str, agent):
        self.description = description
        self.expected_output = expected_output
        self.agent = agent
        self.result = None
    
    def execute(self):
        self.result = self.agent.execute_task(self.description)
        return self.result

class Crew:
    def __init__(self, agents: list, tasks: list, process='sequential'):
        self.agents = agents
        self.tasks = tasks
        self.process = process
    
    def kickoff(self):
        results = []
        for task in self.tasks:
            result = task.execute()
            results.append({
                'task': task.description[:50],
                'agent': task.agent.role,
                'result': result
            })
        return {'results': results, 'process': self.process}

# Example: Research Crew
print('CrewAI Workflow Demo')
print('=' * 80)

researcher = CrewAgent('Researcher', 'Find information', 'Expert researcher', ['web_search'])
writer = CrewAgent('Writer', 'Create reports', 'Technical writer', ['markdown'])
editor = CrewAgent('Editor', 'Refine content', 'Quality editor', ['grammar_check'])

task1 = Task('Research LLM agentic trends', '5 key trends', researcher)
task2 = Task('Write technical report', '2-page report', writer)
task3 = Task('Edit and polish', 'Final report', editor)

crew = Crew([researcher, writer, editor], [task1, task2, task3])
output = crew.kickoff()

print(f'\nProcess: {output["process"]}\n')
for i, res in enumerate(output['results'], 1):
    print(f'Task {i}: {res["task"]}...')
    print(f'  Agent: {res["agent"]}')
    print(f'  Result: {res["result"]}\n')

### Section 2: Hierarchical Process with Manager Agent

Hierarchical crews use a manager to delegate tasks dynamically:
- **Manager**: Assigns tasks based on agent capabilities
- **Specialists**: Execute assigned tasks
- **Quality Control**: Manager reviews outputs
- **Adaptive**: Manager can reassign based on performance

In [ ]:
from typing import List, Dict, Any, Optional, Callable
import json
from dataclasses import dataclass
from datetime import datetime

@dataclass
class AgentCapability:
    '''Agent's capabilities and expertise level'''
    domain: str
    expertise_level: float  # 0-1
    max_concurrent_tasks: int
    avg_completion_time_sec: float

class EnhancedCrewAgent:
    '''Agent with capabilities and performance tracking'''
    
    def __init__(self, 
                 role: str, 
                 goal: str, 
                 backstory: str,
                 capabilities: List[AgentCapability],
                 tools: List[str] = None):
        self.role = role
        self.goal = goal
        self.backstory = backstory
        self.capabilities = {c.domain: c for c in capabilities}
        self.tools = tools or []
        
        # Performance tracking
        self.tasks_completed = 0
        self.tasks_failed = 0
        self.total_time_sec = 0
        self.current_tasks = []
    
    def can_handle(self, task_domain: str, required_expertise: float = 0.5) -> bool:
        '''Check if agent can handle task'''
        if task_domain not in self.capabilities:
            return False
        
        cap = self.capabilities[task_domain]
        
        # Check expertise level
        if cap.expertise_level < required_expertise:
            return False
        
        # Check capacity
        if len(self.current_tasks) >= cap.max_concurrent_tasks:
            return False
        
        return True
    
    def execute_task(self, task: 'EnhancedTask') -> dict:
        '''Execute task with performance tracking'''
        import time
        
        start = time.time()
        
        try:
            # Simulate work
            time.sleep(0.1)  # Mock execution
            
            result = f'[{self.role}] completed: {task.description[:50]}...'
            
            elapsed = time.time() - start
            self.tasks_completed += 1
            self.total_time_sec += elapsed
            
            return {
                'status': 'success',
                'result': result,
                'time_sec': elapsed,
                'agent': self.role
            }
            
        except Exception as e:
            self.tasks_failed += 1
            return {
                'status': 'error',
                'error': str(e),
                'agent': self.role
            }
    
    def get_performance_metrics(self) -> dict:
        '''Get agent performance metrics'''
        total = self.tasks_completed + self.tasks_failed
        return {
            'role': self.role,
            'tasks_completed': self.tasks_completed,
            'tasks_failed': self.tasks_failed,
            'success_rate': self.tasks_completed / total if total > 0 else 0,
            'avg_time_sec': self.total_time_sec / self.tasks_completed if self.tasks_completed > 0 else 0
        }

class EnhancedTask:
    '''Task with domain, priority, and dependencies'''
    
    def __init__(self,
                 description: str,
                 expected_output: str,
                 domain: str,
                 priority: int = 5,  # 1-10
                 required_expertise: float = 0.5,
                 dependencies: List[str] = None):
        self.task_id = f'task_{datetime.utcnow().timestamp()}'
        self.description = description
        self.expected_output = expected_output
        self.domain = domain
        self.priority = priority
        self.required_expertise = required_expertise
        self.dependencies = dependencies or []
        self.assigned_agent = None
        self.result = None
        self.status = 'pending'

class ManagerAgent:
    '''Manager that delegates tasks to specialists'''
    
    def __init__(self, agents: List[EnhancedCrewAgent]):
        self.agents = agents
        self.task_queue = []
        self.completed_tasks = {}
    
    def assign_task(self, task: EnhancedTask) -> Optional[EnhancedCrewAgent]:
        '''Intelligently assign task to best available agent'''
        
        # Find capable agents
        capable_agents = [
            agent for agent in self.agents
            if agent.can_handle(task.domain, task.required_expertise)
        ]
        
        if not capable_agents:
            print(f'No agent capable of: {task.description[:50]}...')
            return None
        
        # Select best agent based on:
        # 1. Expertise level
        # 2. Current workload
        # 3. Past performance
        def score_agent(agent: EnhancedCrewAgent) -> float:
            cap = agent.capabilities[task.domain]
            metrics = agent.get_performance_metrics()
            
            score = (
                cap.expertise_level * 0.4 +  # Expertise
                metrics['success_rate'] * 0.3 +  # Past performance
                (1 - len(agent.current_tasks) / cap.max_concurrent_tasks) * 0.3  # Availability
            )
            
            return score
        
        best_agent = max(capable_agents, key=score_agent)
        task.assigned_agent = best_agent
        
        return best_agent
    
    def execute_crew(self, tasks: List[EnhancedTask]) -> dict:
        '''Execute tasks with hierarchical delegation'''
        
        # Sort by priority and dependencies
        sorted_tasks = self._sort_tasks(tasks)
        
        results = []
        
        for task in sorted_tasks:
            # Check dependencies
            if not self._dependencies_met(task):
                print(f'Waiting for dependencies: {task.description[:50]}...')
                continue
            
            # Assign to agent
            agent = self.assign_task(task)
            
            if agent:
                print(f'Assigning to {agent.role}: {task.description[:50]}...')
                
                # Execute
                result = agent.execute_task(task)
                
                task.result = result
                task.status = result['status']
                self.completed_tasks[task.task_id] = task
                
                results.append({
                    'task_id': task.task_id,
                    'description': task.description[:50],
                    'agent': agent.role,
                    'status': result['status'],
                    'time_sec': result['time_sec']
                })
            else:
                results.append({
                    'task_id': task.task_id,
                    'description': task.description[:50],
                    'status': 'no_agent_available'
                })
        
        return {
            'results': results,
            'total_tasks': len(tasks),
            'successful': sum(1 for r in results if r['status'] == 'success'),
            'failed': sum(1 for r in results if r['status'] in ['error', 'no_agent_available'])
        }
    
    def _sort_tasks(self, tasks: List[EnhancedTask]) -> List[EnhancedTask]:
        '''Sort tasks by priority and dependencies'''
        # Simple sort by priority (higher first)
        return sorted(tasks, key=lambda t: t.priority, reverse=True)
    
    def _dependencies_met(self, task: EnhancedTask) -> bool:
        '''Check if task dependencies are completed'''
        for dep_id in task.dependencies:
            if dep_id not in self.completed_tasks:
                return False
            if self.completed_tasks[dep_id].status != 'success':
                return False
        return True

# Demo: Hierarchical crew
print('HIERARCHICAL CREW WITH MANAGER')
print('=' * 90)

# Create specialist agents
researcher = EnhancedCrewAgent(
    role='Senior Researcher',
    goal='Conduct thorough research',
    backstory='PhD in CS with 10 years research experience',
    capabilities=[
        AgentCapability('research', expertise_level=0.9, max_concurrent_tasks=3, avg_completion_time_sec=60),
        AgentCapability('analysis', expertise_level=0.7, max_concurrent_tasks=2, avg_completion_time_sec=45),
    ],
    tools=['web_search', 'arxiv_search', 'scholar_search']
)

data_analyst = EnhancedCrewAgent(
    role='Data Analyst',
    goal='Analyze data and extract insights',
    backstory='Expert in statistical analysis and visualization',
    capabilities=[
        AgentCapability('analysis', expertise_level=0.95, max_concurrent_tasks=5, avg_completion_time_sec=30),
        AgentCapability('visualization', expertise_level=0.8, max_concurrent_tasks=3, avg_completion_time_sec=20),
    ],
    tools=['pandas', 'matplotlib', 'statsmodels']
)

writer = EnhancedCrewAgent(
    role='Technical Writer',
    goal='Create clear documentation',
    backstory='10 years writing technical content',
    capabilities=[
        AgentCapability('writing', expertise_level=0.9, max_concurrent_tasks=4, avg_completion_time_sec=40),
        AgentCapability('editing', expertise_level=0.85, max_concurrent_tasks=5, avg_completion_time_sec=20),
    ],
    tools=['markdown', 'grammar_check']
)

# Create manager
manager = ManagerAgent([researcher, data_analyst, writer])

# Define tasks
tasks = [
    EnhancedTask(
        'Research latest LLM architectures',
        'Summary of 5 recent architectures',
        domain='research',
        priority=8,
        required_expertise=0.7
    ),
    EnhancedTask(
        'Analyze performance benchmarks',
        'Statistical analysis with visualizations',
        domain='analysis',
        priority=7,
        required_expertise=0.8
    ),
    EnhancedTask(
        'Write technical blog post',
        '2000-word article',
        domain='writing',
        priority=6,
        required_expertise=0.6
    ),
]

# Execute
result = manager.execute_crew(tasks)

print(f'\nResults:')
print(f'  Total tasks: {result["total_tasks"]}')
print(f'  Successful: {result["successful"]}')
print(f'  Failed: {result["failed"]}')

print(f'\nTask breakdown:')
for r in result['results']:
    print(f'  {r["description"]}...')
    print(f'    Agent: {r["agent"]}')
    print(f'    Status: {r["status"]}')
    print(f'    Time: {r.get("time_sec", 0):.2f}s')

print('\n' + '=' * 90)
print('Agent Performance:')
for agent in manager.agents:
    metrics = agent.get_performance_metrics()
    print(f'  {metrics["role"]}: {metrics["tasks_completed"]} tasks, {metrics["success_rate"]:.0%} success, {metrics["avg_time_sec"]:.1f}s avg')

### Section 3: Memory Management in CrewAI

CrewAI supports different memory types:
- **Short-term**: Current task context
- **Long-term**: Persistent facts and learnings
- **Entity memory**: Track entities across tasks
- **Shared memory**: All agents can access/update

In [ ]:
from collections import defaultdict
import hashlib

class CrewMemory:
    '''Memory system for CrewAI workflows'''
    
    def __init__(self):
        self.short_term = []  # Recent messages
        self.long_term = {}   # Key facts
        self.entity_memory = defaultdict(dict)  # Entity tracking
        self.shared_context = {}  # Shared across agents
        self.max_short_term_items = 20
    
    def add_short_term(self, item: dict):
        '''Add to short-term memory (sliding window)'''
        self.short_term.append(item)
        
        # Prune if too long
        if len(self.short_term) > self.max_short_term_items:
            self.short_term.pop(0)
    
    def add_long_term(self, key: str, value: Any):
        '''Store important fact in long-term memory'''
        self.long_term[key] = {
            'value': value,
            'stored_at': datetime.utcnow().isoformat(),
            'access_count': 0
        }
    
    def get_long_term(self, key: str) -> Optional[Any]:
        '''Retrieve from long-term memory'''
        if key in self.long_term:
            self.long_term[key]['access_count'] += 1
            return self.long_term[key]['value']
        return None
    
    def track_entity(self, entity_name: str, attributes: dict):
        '''Track entity across conversation'''
        if entity_name not in self.entity_memory:
            self.entity_memory[entity_name] = {
                'first_mentioned': datetime.utcnow().isoformat(),
                'attributes': {}
            }
        
        # Update attributes
        self.entity_memory[entity_name]['attributes'].update(attributes)
        self.entity_memory[entity_name]['last_updated'] = datetime.utcnow().isoformat()
    
    def get_context_for_agent(self, agent_role: str) -> dict:
        '''Get relevant context for agent'''
        return {
            'recent_messages': self.short_term[-5:],
            'relevant_facts': self._get_relevant_facts(agent_role),
            'shared_context': self.shared_context,
        }
    
    def _get_relevant_facts(self, agent_role: str) -> dict:
        '''Filter long-term memory for relevance'''
        # In production, use embedding similarity
        # For demo, return all
        return {k: v['value'] for k, v in self.long_term.items()}
    
    def summarize(self) -> dict:
        '''Get memory statistics'''
        return {
            'short_term_items': len(self.short_term),
            'long_term_facts': len(self.long_term),
            'tracked_entities': len(self.entity_memory),
            'shared_keys': len(self.shared_context),
        }

class MemoryAwareCrew:
    '''Crew with persistent memory across tasks'''
    
    def __init__(self, agents: List[EnhancedCrewAgent], memory: CrewMemory = None):
        self.agents = agents
        self.memory = memory or CrewMemory()
        self.execution_history = []
    
    def execute_task_sequence(self, tasks: List[EnhancedTask]) -> dict:
        '''Execute tasks with memory'''
        results = []
        
        for task in tasks:
            # Provide memory context to agent
            agent = self._select_agent(task)
            
            if agent:
                context = self.memory.get_context_for_agent(agent.role)
                
                # Add context to task
                task.context = context
                
                # Execute
                result = agent.execute_task(task)
                
                # Store in memory
                self.memory.add_short_term({
                    'task': task.description,
                    'agent': agent.role,
                    'result': result['result'] if result['status'] == 'success' else None
                })
                
                # Extract entities and facts (simplified)
                if result['status'] == 'success':
                    self._extract_and_store(result['result'])
                
                results.append(result)
        
        return {
            'results': results,
            'memory_summary': self.memory.summarize()
        }
    
    def _select_agent(self, task: EnhancedTask) -> Optional[EnhancedCrewAgent]:
        '''Select agent for task'''
        for agent in self.agents:
            if agent.can_handle(task.domain, task.required_expertise):
                return agent
        return None
    
    def _extract_and_store(self, result: str):
        '''Extract facts and entities from result'''
        # Simplified extraction (in production, use NER and fact extraction)
        
        # Store key facts
        if 'important' in result.lower():
            fact_key = hashlib.md5(result.encode()).hexdigest()[:8]
            self.memory.add_long_term(fact_key, result[:100])

# Demo memory-aware crew
print('\nMEMORY-AWARE CREW DEMONSTRATION')
print('=' * 90)

memory = CrewMemory()
crew = MemoryAwareCrew([researcher, data_analyst, writer], memory)

tasks = [
    EnhancedTask('Research transformer architecture', 'Technical summary', 'research', priority=9),
    EnhancedTask('Analyze attention mechanisms', 'Statistical analysis', 'analysis', priority=8),
    EnhancedTask('Write article on findings', 'Blog post', 'writing', priority=7),
]

result = crew.execute_task_sequence(tasks)

print(f'\nMemory Summary: {result["memory_summary"]}')
print(f'  Short-term: {result["memory_summary"]["short_term_items"]} items')
print(f'  Long-term: {result["memory_summary"]["long_term_facts"]} facts')

print('\n' + '=' * 90)
print('KEY MEMORY BENEFITS:')
print('  - Agents share context and learnings')
print('  - Avoid repeating same research')
print('  - Build knowledge base over time')
print('  - Better coherence across tasks')

### Section 4: Tool Integration

CrewAI agents can use tools:
- **Web search**: Find information online
- **File operations**: Read/write files
- **APIs**: Call external services
- **Database**: Query data
- **Custom tools**: Domain-specific operations

In [ ]:
from typing import Callable, Any
import json

class Tool:
    '''Tool that agents can use'''
    
    def __init__(self, name: str, description: str, func: Callable, requires_approval: bool = False):
        self.name = name
        self.description = description
        self.func = func
        self.requires_approval = requires_approval
        self.usage_count = 0
        self.error_count = 0
    
    def execute(self, *args, **kwargs) -> dict:
        '''Execute tool with tracking'''
        self.usage_count += 1
        
        try:
            result = self.func(*args, **kwargs)
            return {
                'status': 'success',
                'result': result,
                'tool': self.name
            }
        except Exception as e:
            self.error_count += 1
            return {
                'status': 'error',
                'error': str(e),
                'tool': self.name
            }
    
    def get_metrics(self) -> dict:
        '''Get tool usage metrics'''
        return {
            'tool': self.name,
            'usage_count': self.usage_count,
            'error_count': self.error_count,
            'success_rate': (self.usage_count - self.error_count) / self.usage_count if self.usage_count > 0 else 0
        }

# Example tools
def web_search_tool(query: str) -> List[str]:
    '''Mock web search'''
    # In production: call Google, Bing, or specialized search API
    return [
        f'Result 1 for {query}',
        f'Result 2 for {query}',
        f'Result 3 for {query}',
    ]

def file_write_tool(filename: str, content: str) -> str:
    '''Mock file write'''
    # In production: actual file I/O with safety checks
    return f'Wrote {len(content)} chars to {filename}'

def api_call_tool(endpoint: str, params: dict) -> dict:
    '''Mock API call'''
    # In production: actual HTTP request
    return {'status': 'success', 'data': f'Response from {endpoint}'}

class ToolEnabledAgent(EnhancedCrewAgent):
    '''Agent that can use tools'''
    
    def __init__(self, *args, tool_objects: List[Tool] = None, **kwargs):
        super().__init__(*args, **kwargs)
        self.tool_objects = {t.name: t for t in (tool_objects or [])}
        self.tool_usage_log = []
    
    def use_tool(self, tool_name: str, *args, **kwargs) -> dict:
        '''Use a tool'''
        if tool_name not in self.tool_objects:
            return {'status': 'error', 'error': f'Tool {tool_name} not available'}
        
        tool = self.tool_objects[tool_name]
        
        # Check if approval required
        if tool.requires_approval:
            print(f'  Tool {tool_name} requires approval...')
            # In production: request approval
        
        # Execute tool
        result = tool.execute(*args, **kwargs)
        
        # Log usage
        self.tool_usage_log.append({
            'tool': tool_name,
            'timestamp': datetime.utcnow().isoformat(),
            'status': result['status']
        })
        
        return result

# Create tools
tools = [
    Tool('web_search', 'Search the web for information', web_search_tool),
    Tool('file_write', 'Write content to file', file_write_tool, requires_approval=True),
    Tool('api_call', 'Call external API', api_call_tool),
]

# Create tool-enabled agent
research_agent = ToolEnabledAgent(
    role='Research Agent',
    goal='Gather information',
    backstory='Expert researcher',
    capabilities=[AgentCapability('research', 0.9, 3, 30)],
    tool_objects=tools
)

print('\nTOOL INTEGRATION DEMONSTRATION')
print('=' * 90)

# Use tools
print('\nAgent using tools:')
result1 = research_agent.use_tool('web_search', 'LLM agent frameworks')
print(f'  web_search: {result1["status"]}')

result2 = research_agent.use_tool('file_write', 'report.md', 'Research findings...')
print(f'  file_write: {result2["status"]}')

# Check tool metrics
print('\nTool Usage Metrics:')
for tool in tools:
    metrics = tool.get_metrics()
    print(f'  {metrics["tool"]}: {metrics["usage_count"]} calls, {metrics["success_rate"]:.0%} success')

print('\n' + '=' * 90)

### Section 5: Quality Control and Validation

Production crews need output validation:
- **Schema validation**: Ensure output matches expected format
- **Quality scoring**: Automatic quality assessment
- **Human review**: For critical outputs
- **Revision loops**: Re-do if quality too low

In [ ]:
from pydantic import BaseModel, Field, validator
from typing import Literal

class TaskOutput(BaseModel):
    '''Validated task output'''
    content: str = Field(min_length=100, description='Main content')
    quality_score: float = Field(ge=0.0, le=1.0, description='Quality score')
    citations: List[str] = Field(default_factory=list, description='Sources cited')
    confidence: float = Field(ge=0.0, le=1.0, description='Agent confidence')
    requires_review: bool = Field(default=False, description='Needs human review')
    
    @validator('quality_score')
    def check_quality(cls, v, values):
        '''Validate quality meets minimum threshold'''
        if v < 0.7:
            raise ValueError(f'Quality score {v} below threshold 0.7')
        return v

class QualityController:
    '''Quality control for crew outputs'''
    
    def __init__(self, min_quality_score=0.7, enable_revision=True):
        self.min_quality_score = min_quality_score
        self.enable_revision = enable_revision
        self.revision_history = []
    
    def assess_quality(self, output: str, expected_criteria: dict) -> float:
        '''Assess output quality (simplified)'''
        score = 0.5  # Baseline
        
        # Length check
        if len(output) >= expected_criteria.get('min_length', 100):
            score += 0.15
        
        # Keyword presence
        required_keywords = expected_criteria.get('keywords', [])
        keyword_coverage = sum(1 for kw in required_keywords if kw.lower() in output.lower())
        score += (keyword_coverage / len(required_keywords)) * 0.2 if required_keywords else 0.1
        
        # Structure check (has sections)
        if '\n\n' in output:  # Has paragraphs
            score += 0.1
        
        # Citations present
        if '[' in output and ']' in output:  # Has citations
            score += 0.05
        
        return min(score, 1.0)
    
    def validate_output(self, output: str, expected_criteria: dict, max_revisions=2) -> dict:
        '''Validate output with optional revision loop'''
        
        revision_count = 0
        current_output = output
        
        while revision_count <= max_revisions:
            # Assess quality
            quality_score = self.assess_quality(current_output, expected_criteria)
            
            # Check if meets threshold
            if quality_score >= self.min_quality_score:
                return {
                    'status': 'approved',
                    'output': current_output,
                    'quality_score': quality_score,
                    'revisions': revision_count
                }
            
            # Try revision
            if self.enable_revision and revision_count < max_revisions:
                print(f'Quality score {quality_score:.2f} below threshold, requesting revision...')
                
                # Request revision (in production: agent revises)
                feedback = self._generate_feedback(current_output, quality_score, expected_criteria)
                current_output = self._revise_output(current_output, feedback)
                
                revision_count += 1
                
                self.revision_history.append({
                    'iteration': revision_count,
                    'quality_score': quality_score,
                    'feedback': feedback
                })
            else:
                break
        
        # Failed quality check
        return {
            'status': 'failed',
            'output': current_output,
            'quality_score': quality_score,
            'revisions': revision_count,
            'requires_human_review': True
        }
    
    def _generate_feedback(self, output: str, score: float, criteria: dict) -> str:
        '''Generate improvement feedback'''
        feedback = []
        
        if len(output) < criteria.get('min_length', 100):
            feedback.append('Output too short, needs more detail')
        
        required_kw = criteria.get('keywords', [])
        missing_kw = [kw for kw in required_kw if kw.lower() not in output.lower()]
        if missing_kw:
            feedback.append(f'Missing keywords: {missing_kw}')
        
        if '[' not in output:
            feedback.append('Add citations to support claims')
        
        return '; '.join(feedback)
    
    def _revise_output(self, output: str, feedback: str) -> str:
        '''Revise output based on feedback'''
        # In production: agent revises
        # For demo: simple improvement
        revised = output + '\n\nAdditional details and [citation needed].'
        return revised

class QualityAwareCrew:
    '''Crew with automatic quality control'''
    
    def __init__(self, agents: List[EnhancedCrewAgent]):
        self.agents = agents
        self.quality_controller = QualityController(min_quality_score=0.7)
    
    def execute_with_qc(self, task: EnhancedTask, expected_criteria: dict) -> dict:
        '''Execute task with quality control'''
        
        # Select agent
        agent = None
        for a in self.agents:
            if a.can_handle(task.domain, task.required_expertise):
                agent = a
                break
        
        if not agent:
            return {'status': 'no_agent'}
        
        # Execute task
        result = agent.execute_task(task)
        
        if result['status'] != 'success':
            return result
        
        # Quality control
        qc_result = self.quality_controller.validate_output(
            result['result'],
            expected_criteria,
            max_revisions=2
        )
        
        return {
            **result,
            'qc_status': qc_result['status'],
            'quality_score': qc_result['quality_score'],
            'revisions': qc_result['revisions']
        }

# Demo quality control
print('\nQUALITY CONTROL DEMONSTRATION')
print('=' * 90)

qc_crew = QualityAwareCrew([writer])

task = EnhancedTask(
    'Write article about RAG systems',
    'Technical article',
    domain='writing',
    required_expertise=0.6
)

criteria = {
    'min_length': 200,
    'keywords': ['retrieval', 'generation', 'embeddings'],
}

result = qc_crew.execute_with_qc(task, criteria)

print(f'\nExecution Result:')
print(f'  Status: {result["status"]}')
print(f'  QC Status: {result["qc_status"]}')
print(f'  Quality Score: {result["quality_score"]:.2f}')
print(f'  Revisions: {result["revisions"]}')

print('\n' + '=' * 90)
print('KEY QC PATTERNS:')
print('  - Automatic quality assessment')
print('  - Revision loops for improvement')
print('  - Schema validation (Pydantic)')
print('  - Human escalation for low quality')
print('  - Track quality metrics over time')

## Interview Questions: CrewAI Production Systems

### For Senior/Staff Engineers

In [ ]:
crewai_interview_questions = [
    {
        'level': 'Senior',
        'question': 'You have a research crew (Researcher → Analyst → Writer → Editor). The Researcher takes 5 minutes, bottlenecking the entire pipeline. How do you optimize while maintaining output quality?',
        'answer': '''
**Problem Analysis:**
- Current: Sequential execution
- Bottleneck: Researcher (5 min)
- Total time: 5 + 2 + 3 + 1 = 11 minutes
- Throughput: ~5.5 reports/hour
- Writer and Editor idle for first 7 minutes

**Optimization Strategies:**

**1. Parallel Research (Best: 60% reduction)**
```python
class ParallelResearchCrew:
    '''Split research among multiple specialists'''
    
    def __init__(self):
        # Multiple researchers for different aspects
        self.researchers = [
            Agent('Researcher_Academic', expertise=['papers', 'research']),
            Agent('Researcher_Industry', expertise=['products', 'companies']),
            Agent('Researcher_News', expertise=['trends', 'events']),
        ]
        self.analyst = Agent('Analyst')
        self.writer = Agent('Writer')
        self.editor = Agent('Editor')
    
    async def execute(self, topic: str):
        # Parallel research (5 min → 2 min with 3 researchers)
        research_tasks = [
            researcher.research(f'{topic} - {researcher.expertise}')
            for researcher in self.researchers
        ]
        research_results = await asyncio.gather(*research_tasks)
        
        # Consolidate research (2 min)
        analysis = await self.analyst.analyze(research_results)
        
        # Sequential writing and editing (3 + 1 = 4 min)
        draft = await self.writer.write(analysis)
        final = await self.editor.edit(draft)
        
        return final

# Time: max(2,2,2) + 2 + 3 + 1 = 8 min (27% faster)
```

**2. Pipeline with Batching (45% throughput increase)**
```python
class PipelinedCrew:
    '''Process multiple reports in pipeline'''
    
    def __init__(self):
        self.researcher = Agent('Researcher')
        self.analyst = Agent('Analyst')
        self.writer = Agent('Writer')
        self.editor = Agent('Editor')
        
        # Queues between stages
        self.research_queue = Queue()
        self.analysis_queue = Queue()
        self.writing_queue = Queue()
    
    async def run_pipeline(self, topics: List[str]):
        '''Run pipeline for multiple topics'''
        
        # Start all stages concurrently
        await asyncio.gather(
            self._research_stage(topics),
            self._analysis_stage(),
            self._writing_stage(),
            self._editing_stage()
        )
    
    async def _research_stage(self, topics):
        for topic in topics:
            result = await self.researcher.research(topic)  # 5 min
            await self.research_queue.put(result)
    
    async def _analysis_stage(self):
        while True:
            research = await self.research_queue.get()
            analysis = await self.analyst.analyze(research)  # 2 min
            await self.analysis_queue.put(analysis)
    
    # ... similar for other stages

# Throughput with pipeline:
# Report 1: 11 min
# Report 2: 11 min (starts while Report 1 in progress)
# Steady state: 5 reports/5 min = 60 reports/hour vs 33/hour (82% improvement)
```

**3. Caching Research (50% improvement for repeated topics)**
```python
import hashlib

class CachedResearcher:
    '''Cache research results'''
    
    def __init__(self, cache_ttl_hours=24):
        self.cache = {}
        self.cache_ttl = cache_ttl_hours * 3600
    
    async def research(self, topic: str) -> dict:
        # Generate cache key
        cache_key = hashlib.md5(topic.lower().encode()).hexdigest()
        
        # Check cache
        if cache_key in self.cache:
            cached = self.cache[cache_key]
            if time.time() - cached['timestamp'] < self.cache_ttl:
                print(f'Cache hit for: {topic}')
                return cached['result']
        
        # Perform research
        result = await self._do_research(topic)  # 5 min
        
        # Cache result
        self.cache[cache_key] = {
            'result': result,
            'timestamp': time.time()
        }
        
        return result

# With 40% cache hit rate: 5 min → 3 min average (40% improvement)
```

**4. Adaptive Depth (30% improvement)**
```python
class AdaptiveResearcher:
    '''Adjust research depth based on requirements'''
    
    def research(self, topic: str, depth: str = 'standard'):
        if depth == 'quick':
            # Surface-level: 1 min
            return self.quick_research(topic)
        elif depth == 'standard':
            # Normal: 5 min
            return self.standard_research(topic)
        elif depth == 'deep':
            # Comprehensive: 15 min
            return self.deep_research(topic)
    
    def determine_required_depth(self, task_metadata: dict) -> str:
        '''Decide research depth needed'''
        if task_metadata.get('audience') == 'executive':
            return 'quick'  # High-level summary
        elif task_metadata.get('technical_depth') == 'high':
            return 'deep'  # Detailed technical analysis
        else:
            return 'standard'
```

**5. Combined Solution:**
```python
class OptimizedCrew:
    def __init__(self):
        # Multiple parallel researchers with caching
        self.researchers = [
            CachedResearcher('Academic'),
            CachedResearcher('Industry'),
            CachedResearcher('News'),
        ]
        
        # Adaptive depth selection
        self.depth_controller = AdaptiveResearcher()
        
        # Pipeline for batch processing
        self.pipeline = PipelinedCrew()
    
    async def execute_optimized(self, topic: str, metadata: dict):
        # Determine depth
        depth = self.depth_controller.determine_required_depth(metadata)
        
        # Parallel research with caching
        research_results = await asyncio.gather(*[
            researcher.research(topic, depth=depth)
            for researcher in self.researchers
        ])
        
        # Rest of pipeline...
        return self._complete_pipeline(research_results)
```

**Results:**

| Approach | Time | Throughput | Improvement |
|----------|------|------------|-------------|
| Baseline | 11 min | 5.5/hour | - |
| Parallel Research | 8 min | 7.5/hour | +36% |
| Pipeline (batch) | 11 min first, 5 min steady | 12/hour | +118% |
| Caching (40% hit) | 8.2 min avg | 7.3/hour | +33% |
| **Combined** | **5 min steady state** | **12/hour** | **+118%** |

**Trade-offs:**
- Parallel research: Slightly less deep but faster
- Pipeline: Higher throughput but more complex
- Caching: Staleness risk (set appropriate TTL)
- Adaptive depth: May miss details on 'quick' mode

**Recommendation:**
- Use combined approach for production
- Parallel research for speed
- Caching with 24-hour TTL
- Pipeline for batch processing
- Adaptive depth based on use case
- Monitor cache hit rate (target: 40%+)
- A/B test quality: ensure optimizations don't degrade output
        ''',
    },
    {
        'level': 'Staff',
        'question': 'Design a CrewAI system that produces a daily market intelligence report by coordinating 10+ specialist agents. Include task dependencies, parallel execution, quality control, and human review integration. System must complete in under 30 minutes.',
        'answer': '''
**Market Intelligence System Architecture:**

**1. Agent Team Structure (10 specialists)**
```python
agent_team = {
    # Research tier (parallel execution)
    'news_researcher': {
        'focus': 'Breaking news, press releases',
        'sources': ['news_api', 'rss_feeds'],
        'time': '5 min',
    },
    'social_researcher': {
        'focus': 'Social media trends, sentiment',
        'sources': ['twitter_api', 'reddit_api'],
        'time': '5 min',
    },
    'market_researcher': {
        'focus': 'Stock data, financial reports',
        'sources': ['yahoo_finance', 'sec_filings'],
        'time': '5 min',
    },
    'competitor_researcher': {
        'focus': 'Competitor analysis',
        'sources': ['crunchbase', 'product_hunt'],
        'time': '5 min',
    },
    
    # Analysis tier (depends on research)
    'data_analyst': {
        'focus': 'Quantitative analysis',
        'time': '3 min',
    },
    'sentiment_analyst': {
        'focus': 'Sentiment and narrative analysis',
        'time': '3 min',
    },
    
    # Synthesis tier
    'synthesizer': {
        'focus': 'Combine all inputs into coherent narrative',
        'time': '4 min',
    },
    
    # Writing tier
    'writer': {
        'focus': 'Draft report sections',
        'time': '5 min',
    },
    
    # Review tier
    'fact_checker': {
        'focus': 'Verify claims and citations',
        'time': '3 min',
    },
    'editor': {
        'focus': 'Polish and format final report',
        'time': '2 min',
    },
}

# Total sequential: 40 min (exceeds 30 min limit)
# Need parallelization!
```

**2. Dependency Graph with Parallel Execution:**
```python
import networkx as nx

class TaskDAG:
    '''Directed Acyclic Graph for task dependencies'''
    
    def __init__(self):
        self.graph = nx.DiGraph()
        self.task_metadata = {}
    
    def add_task(self, task_id: str, agent: str, duration_min: float, dependencies: List[str] = None):
        '''Add task to DAG'''
        self.graph.add_node(task_id)
        self.task_metadata[task_id] = {
            'agent': agent,
            'duration_min': duration_min,
            'status': 'pending'
        }
        
        # Add dependency edges
        for dep in (dependencies or []):
            self.graph.add_edge(dep, task_id)
    
    def get_execution_levels(self) -> List[List[str]]:
        '''Get tasks grouped by dependency level (can execute in parallel)'''
        return list(nx.topological_generations(self.graph))
    
    def get_critical_path(self) -> Tuple[List[str], float]:
        '''Find critical path (longest path through DAG)'''
        # Calculate longest path
        levels = self.get_execution_levels()
        
        critical_path = []
        total_time = 0
        
        for level in levels:
            # Within each level, find longest task
            longest_task = max(level, key=lambda t: self.task_metadata[t]['duration_min'])
            critical_path.append(longest_task)
            total_time += self.task_metadata[longest_task]['duration_min']
        
        return critical_path, total_time

# Build task DAG
dag = TaskDAG()

# Research tier (can run in parallel)
dag.add_task('research_news', 'news_researcher', 5.0)
dag.add_task('research_social', 'social_researcher', 5.0)
dag.add_task('research_market', 'market_researcher', 5.0)
dag.add_task('research_competitor', 'competitor_researcher', 5.0)

# Analysis tier (depends on research)
dag.add_task('analyze_data', 'data_analyst', 3.0, 
            dependencies=['research_market', 'research_competitor'])
dag.add_task('analyze_sentiment', 'sentiment_analyst', 3.0,
            dependencies=['research_news', 'research_social'])

# Synthesis (depends on all analysis)
dag.add_task('synthesize', 'synthesizer', 4.0,
            dependencies=['analyze_data', 'analyze_sentiment'])

# Writing (depends on synthesis)
dag.add_task('write_report', 'writer', 5.0,
            dependencies=['synthesize'])

# Review (depends on writing)
dag.add_task('fact_check', 'fact_checker', 3.0,
            dependencies=['write_report'])
dag.add_task('edit', 'editor', 2.0,
            dependencies=['fact_check'])

# Analyze execution
levels = dag.get_execution_levels()
critical_path, critical_time = dag.get_critical_path()

print('\nTASK EXECUTION PLAN')
print('=' * 80)
for i, level in enumerate(levels, 1):
    max_time = max(dag.task_metadata[t]['duration_min'] for t in level)
    print(f'\nLevel {i} (parallel, {max_time} min):')
    for task_id in level:
        meta = dag.task_metadata[task_id]
        print(f'  - {task_id}: {meta["agent"]} ({meta["duration_min"]} min)')

print(f'\n{'=' * 80}')
print(f'Critical Path: {" → ".join(critical_path)}')
print(f'Total Time: {critical_time} min')
print(f'Speedup: {40 / critical_time:.1f}x (40 min sequential → {critical_time} min parallel)')
```

**3. Execution Engine with Parallel Processing:**
```python
class ParallelCrewExecutor:
    '''Execute DAG with maximum parallelism'''
    
    def __init__(self, dag: TaskDAG, agents: Dict[str, Agent]):
        self.dag = dag
        self.agents = agents
        self.results = {}
    
    async def execute_dag(self) -> dict:
        '''Execute all tasks respecting dependencies'''
        levels = self.dag.get_execution_levels()
        
        total_start = time.time()
        
        for level_num, level_tasks in enumerate(levels, 1):
            print(f'\nExecuting level {level_num} ({len(level_tasks)} tasks in parallel)...')
            
            level_start = time.time()
            
            # Execute all tasks in level concurrently
            tasks = []
            for task_id in level_tasks:
                meta = self.dag.task_metadata[task_id]
                agent = self.agents[meta['agent']]
                
                task = self._execute_task(task_id, agent)
                tasks.append(task)
            
            # Wait for all tasks in level
            level_results = await asyncio.gather(*tasks)
            
            # Store results
            for task_id, result in zip(level_tasks, level_results):
                self.results[task_id] = result
            
            level_time = time.time() - level_start
            print(f'  Level {level_num} completed in {level_time:.1f}s')
        
        total_time = time.time() - total_start
        
        return {
            'results': self.results,
            'total_time_sec': total_time,
            'levels_executed': len(levels)
        }
    
    async def _execute_task(self, task_id: str, agent: Agent) -> dict:
        '''Execute single task'''
        # Simulate async work
        meta = self.dag.task_metadata[task_id]
        await asyncio.sleep(meta['duration_min'] * 0.01)  # Scaled for demo
        
        return {
            'task_id': task_id,
            'status': 'success',
            'agent': meta['agent']
        }
```

**4. Quality Control Gates:**
```python
class QualityGate:
    '''Quality control between stages'''
    
    def __init__(self, min_quality=0.75):
        self.min_quality = min_quality
    
    async def validate_stage_output(self, output: str, stage: str) -> dict:
        '''Validate output quality before next stage'''
        
        # Calculate quality score
        quality_score = self._assess_quality(output, stage)
        
        if quality_score < self.min_quality:
            return {
                'passed': False,
                'score': quality_score,
                'action': 'revise',
                'feedback': self._generate_feedback(output, stage)
            }
        
        return {
            'passed': True,
            'score': quality_score
        }
    
    def _assess_quality(self, output: str, stage: str) -> float:
        '''Stage-specific quality assessment'''
        score = 0.5
        
        if stage == 'research':
            # Check for citations
            if '[' in output and ']' in output:
                score += 0.2
            # Check for data
            if any(char.isdigit() for char in output):
                score += 0.15
        
        elif stage == 'analysis':
            # Check for insights
            if 'trend' in output.lower() or 'insight' in output.lower():
                score += 0.2
        
        # Length check
        if len(output) > 500:
            score += 0.15
        
        return min(score, 1.0)
```

**5. Human Review Integration:**
```python
class HumanReviewQueue:
    '''Queue high-priority items for human review'''
    
    def __init__(self):
        self.queue = []
        self.reviewed = {}
    
    def submit_for_review(self, report_id: str, report: str, priority: str = 'normal'):
        '''Submit report for human review'''
        self.queue.append({
            'report_id': report_id,
            'report': report,
            'priority': priority,
            'submitted_at': datetime.utcnow().isoformat(),
            'status': 'pending'
        })
        
        # Send notification (Slack, email, etc.)
        if priority == 'high':
            self._send_urgent_notification(report_id)
    
    def get_pending_reviews(self, priority: str = None) -> List[dict]:
        '''Get reports awaiting review'''
        pending = [r for r in self.queue if r['status'] == 'pending']
        
        if priority:
            pending = [r for r in pending if r['priority'] == priority]
        
        return pending
```

**6. Complete System:**
```python
class MarketIntelligenceCrew:
    '''Complete market intelligence system'''
    
    def __init__(self):
        self.dag = self._build_dag()
        self.agents = self._initialize_agents()
        self.executor = ParallelCrewExecutor(self.dag, self.agents)
        self.quality_gates = QualityGate(min_quality=0.75)
        self.human_review = HumanReviewQueue()
        self.cache = CachedResearcher(cache_ttl_hours=24)
    
    async def generate_daily_report(self, date: str) -> dict:
        '''Generate complete market intelligence report'''
        
        print(f'Generating report for {date}...')
        start = time.time()
        
        # Execute DAG
        result = await self.executor.execute_dag()
        
        # Quality check
        final_report = result['results']['edit']
        qc_result = await self.quality_gates.validate_stage_output(final_report, 'final')
        
        elapsed = time.time() - start
        
        if qc_result['passed']:
            print(f'✓ Report generated in {elapsed:.1f}s (quality: {qc_result["score"]:.2f})')
            
            # Auto-publish if quality high enough
            if qc_result['score'] >= 0.9:
                return {'status': 'published', 'report': final_report}
            else:
                # Submit for human review
                self.human_review.submit_for_review(date, final_report, priority='normal')
                return {'status': 'pending_review', 'report': final_report}
        else:
            # Quality too low, escalate
            self.human_review.submit_for_review(date, final_report, priority='high')
            return {'status': 'needs_revision', 'feedback': qc_result['feedback']}
```

**Execution Timeline (Target: < 30 min):**
```
Level 1 (0-5 min): Research [4 agents in parallel]
  └─ news, social, market, competitor research

Level 2 (5-8 min): Analysis [2 agents in parallel]
  └─ data analysis, sentiment analysis

Level 3 (8-12 min): Synthesis [1 agent]
  └─ combine insights

Level 4 (12-17 min): Writing [1 agent]
  └─ draft sections

Level 5 (17-22 min): Review [2 agents in parallel]
  └─ fact check + edit

Total: 22 minutes ✓ (under 30 min target)
```

**7. Monitoring Dashboard:**
```python
class CrewMonitor:
    '''Real-time monitoring of crew execution'''
    
    def __init__(self):
        self.metrics = defaultdict(list)
    
    def track_execution(self, report_date: str, execution_result: dict):
        self.metrics['execution_time'].append(execution_result['total_time_sec'])
        self.metrics['report_date'].append(report_date)
    
    def get_dashboard_data(self) -> dict:
        return {
            'avg_execution_time_min': np.mean(self.metrics['execution_time']) / 60,
            'p95_execution_time_min': np.percentile(self.metrics['execution_time'], 95) / 60,
            'reports_generated': len(self.metrics['report_date']),
            'success_rate': 0.95,  # Track separately
        }
```

**Key Features:**
- ✓ 10 specialist agents
- ✓ 5-level DAG with parallelism
- ✓ Completes in 22 minutes (< 30 min target)
- ✓ Quality gates between stages
- ✓ Auto-publish or human review based on quality
- ✓ Caching for efficiency
- ✓ Monitoring and metrics

**Cost Analysis:**
- 10 agents × 3 min avg × $0.02/min = $0.60 per report
- 365 reports/year × $0.60 = $219/year
- vs. Human analyst: 4 hours × $50/hour = $200/day = $73K/year
- **Savings: 99.7%**
        ''',
    },
]

for i, qa in enumerate(crewai_interview_questions, 1):
    print(f'\n{'=' * 100}')
    print(f'Q{i} [{qa["level"]} Level]')
    print('=' * 100)
    print(f'\n{qa["question"]}\n')
    print('ANSWER:')
    print(qa['answer'])
    print()

## CrewAI Framework - Complete Core Components

### Based on 8-Hour Curriculum

This section covers all CrewAI core components with production implementations:
- Agent (with role, goal, backstory)
- Task (with expected output)
- Crew (orchestration)
- Tool (function integration)
- Process (sequential vs hierarchical)
- Memory (short-term, long-term, entity)
- Output Parser (structured validation)
- Flow (complex workflows)
- Config (YAML-based setup)

### Component 1: Agent - Complete Implementation

CrewAI Agent with all features: role, goal, backstory, tools, delegation.

In [ ]:
from typing import List, Dict, Any, Optional, Callable
from pydantic import BaseModel, Field
import json

class CrewAIAgent:
    '''
    Complete CrewAI Agent implementation.
    
    Core attributes:
    - role: Job title and function
    - goal: What the agent aims to achieve
    - backstory: Context that shapes behavior
    - tools: Functions the agent can use
    - verbose: Logging level
    - allow_delegation: Can delegate to other agents
    - max_iter: Maximum iterations for task
    '''
    
    def __init__(self,
                 role: str,
                 goal: str,
                 backstory: str,
                 tools: List = None,
                 verbose: bool = True,
                 allow_delegation: bool = False,
                 max_iter: int = 15,
                 memory: bool = True):
        
        self.role = role
        self.goal = goal
        self.backstory = backstory
        self.tools = tools or []
        self.verbose = verbose
        self.allow_delegation = allow_delegation
        self.max_iter = max_iter
        self.memory_enabled = memory
        
        # Internal state
        self.task_history = []
        self.tool_usage = defaultdict(int)
        self.delegation_count = 0
    
    def execute_task(self, task: 'CrewAITask', context: dict = None) -> dict:
        '''
        Execute a task with full CrewAI capabilities.
        '''
        
        iterations = 0
        result = None
        
        while iterations < self.max_iter:
            iterations += 1
            
            if self.verbose:
                print(f'  [{self.role}] Iteration {iterations}/{self.max_iter}')
            
            # Attempt to complete task
            try:
                # In production: LLM generates plan and executes
                result = self._work_on_task(task, context)
                
                # Check if task complete
                if self._is_task_complete(result, task.expected_output):
                    break
                
            except Exception as e:
                if self.verbose:
                    print(f'  Error: {e}')
                
                # Try delegation if allowed
                if self.allow_delegation and self.delegation_count < 2:
                    result = self._delegate_task(task)
                    self.delegation_count += 1
                    break
                else:
                    result = {'error': str(e)}
                    break
        
        # Store in history
        self.task_history.append({
            'task': task.description,
            'result': result,
            'iterations': iterations
        })
        
        return {
            'result': result,
            'iterations': iterations,
            'agent': self.role,
            'tools_used': dict(self.tool_usage)
        }
    
    def _work_on_task(self, task, context) -> str:
        '''Work on task (calls LLM in production)'''
        
        # Build prompt from role, goal, backstory, and task
        prompt = f'''Role: {self.role}
Goal: {self.goal}
Backstory: {self.backstory}

Task: {task.description}
Expected Output: {task.expected_output}

Provide your response:'''
        
        # In production: call LLM
        # response = llm.generate(prompt)
        
        # Mock response
        response = f'[{self.role}] completed task: {task.description[:50]}...'
        
        # Use tools if needed
        if self.tools and 'search' in task.description.lower():
            tool_result = self._use_tool('search', task.description)
            response += f'\nTool result: {tool_result}'
        
        return response
    
    def _use_tool(self, tool_name: str, input_data: str) -> str:
        '''Use a tool'''
        self.tool_usage[tool_name] += 1
        
        # Find tool in list
        for tool in self.tools:
            if tool.name == tool_name:
                return tool.run(input_data)
        
        return f'Tool {tool_name} not found'
    
    def _is_task_complete(self, result: Any, expected_output: str) -> bool:
        '''Check if task meets expected output'''
        # In production: Use semantic similarity or LLM evaluation
        return result is not None and len(str(result)) > 50
    
    def _delegate_task(self, task) -> str:
        '''Delegate task to another agent'''
        if self.verbose:
            print(f'  [{self.role}] Delegating task...')
        return f'Task delegated by {self.role}'
    
    def get_performance_summary(self) -> dict:
        '''Get agent performance metrics'''
        return {
            'role': self.role,
            'tasks_completed': len(self.task_history),
            'avg_iterations': np.mean([t['iterations'] for t in self.task_history]) if self.task_history else 0,
            'tools_used': dict(self.tool_usage),
            'delegations': self.delegation_count
        }

print('CREWAI AGENT - COMPLETE IMPLEMENTATION')
print('=' * 90)

# Create production-ready agent
research_agent = CrewAIAgent(
    role='Senior Research Analyst',
    goal='Conduct thorough research and provide data-driven insights',
    backstory='''You are a senior research analyst with 10 years of experience 
in market research and competitive analysis. You excel at finding reliable sources, 
validating information, and synthesizing complex data into actionable insights.''',
    tools=[],  # Will add tools later
    verbose=True,
    allow_delegation=True,
    max_iter=10,
    memory=True
)

print(f'\nAgent Created:')
print(f'  Role: {research_agent.role}')
print(f'  Goal: {research_agent.goal}')
print(f'  Backstory: {research_agent.backstory[:80]}...')
print(f'  Max iterations: {research_agent.max_iter}')
print(f'  Can delegate: {research_agent.allow_delegation}')

print('\n' + '=' * 90)
print('AGENT DESIGN PRINCIPLES:')
print('  - Role: Clear job title and function')
print('  - Goal: Specific, measurable objective')
print('  - Backstory: Adds context and personality')
print('  - Tools: Extend agent capabilities')
print('  - Delegation: Enable collaboration')

### Component 2: Task - With Expected Output and Context

In [ ]:
class CrewAITask:
    '''
    CrewAI Task with complete configuration.
    
    Attributes:
    - description: What needs to be done
    - expected_output: Criteria for completion
    - agent: Assigned agent
    - context: Input from previous tasks
    - tools: Task-specific tools
    - async_execution: Run asynchronously
    - output_format: Structured output specification
    '''
    
    def __init__(self,
                 description: str,
                 expected_output: str,
                 agent: Optional[CrewAIAgent] = None,
                 context: List['CrewAITask'] = None,
                 tools: List = None,
                 async_execution: bool = False,
                 output_format: Optional[BaseModel] = None):
        
        self.task_id = hashlib.md5(description.encode()).hexdigest()[:8]
        self.description = description
        self.expected_output = expected_output
        self.agent = agent
        self.context = context or []  # Depends on these tasks
        self.tools = tools or []
        self.async_execution = async_execution
        self.output_format = output_format
        
        self.result = None
        self.status = 'pending'
        self.execution_time_sec = 0
    
    def execute(self) -> dict:
        '''Execute task'''
        import time
        start = time.time()
        
        # Get context from dependent tasks
        context_data = self._gather_context()
        
        # Execute with agent
        if self.agent:
            result = self.agent.execute_task(self, context_data)
            self.result = result['result']
            self.status = 'completed' if result['result'] else 'failed'
        else:
            self.result = None
            self.status = 'no_agent'
        
        self.execution_time_sec = time.time() - start
        
        # Validate output format if specified
        if self.output_format and self.result:
            try:
                validated = self.output_format.parse_obj(json.loads(str(self.result)))
                self.result = validated
            except Exception as e:
                self.status = 'validation_failed'
        
        return {
            'task_id': self.task_id,
            'status': self.status,
            'result': self.result,
            'execution_time': self.execution_time_sec
        }
    
    def _gather_context(self) -> dict:
        '''Gather results from context tasks'''
        context_data = {}
        for ctx_task in self.context:
            if ctx_task.result:
                context_data[ctx_task.task_id] = ctx_task.result
        return context_data
    
    def __repr__(self):
        return f'Task({self.task_id}, {self.description[:30]}...)'

# Example task with output format
class ResearchOutput(BaseModel):
    '''Structured research output'''
    summary: str = Field(description='Brief summary')
    key_findings: List[str] = Field(description='3-5 key findings')
    sources: List[str] = Field(description='Cited sources')
    confidence: float = Field(ge=0.0, le=1.0)

print('\nCREWAI TASK - COMPLETE IMPLEMENTATION')
print('=' * 90)

# Create tasks with dependencies
task1 = CrewAITask(
    description='Research current trends in LLM agents',
    expected_output='List of 5 trends with sources',
    agent=research_agent,
    output_format=ResearchOutput
)

task2 = CrewAITask(
    description='Analyze the research findings',
    expected_output='Detailed analysis with insights',
    context=[task1],  # Depends on task1
    async_execution=False
)

task3 = CrewAITask(
    description='Write executive summary',
    expected_output='2-paragraph summary',
    context=[task1, task2],  # Depends on both
    async_execution=False
)

print(f'\nTask chain created:')
for i, task in enumerate([task1, task2, task3], 1):
    print(f'  {i}. {task.description}')
    print(f'     Expected: {task.expected_output}')
    if task.context:
        print(f'     Depends on: {[t.task_id for t in task.context]}')

print('\n' + '=' * 90)
print('TASK DESIGN PRINCIPLES:')
print('  - Clear description (what to do)')
print('  - Specific expected output (completion criteria)')
print('  - Dependencies via context parameter')
print('  - Output format for validation')
print('  - Async execution for parallelism')

### Component 3: Crew - Complete Orchestration

In [ ]:
from enum import Enum

class ProcessType(Enum):
    SEQUENTIAL = 'sequential'
    HIERARCHICAL = 'hierarchical'

class CrewAICrew:
    '''
    Complete CrewAI Crew implementation.
    
    Features:
    - Sequential or hierarchical process
    - Task dependency management
    - Agent coordination
    - Memory sharing
    - Output aggregation
    - Progress tracking
    '''
    
    def __init__(self,
                 agents: List[CrewAIAgent],
                 tasks: List[CrewAITask],
                 process: ProcessType = ProcessType.SEQUENTIAL,
                 verbose: bool = True,
                 memory: bool = True,
                 cache: bool = True):
        
        self.agents = agents
        self.tasks = tasks
        self.process = process
        self.verbose = verbose
        self.memory_enabled = memory
        self.cache_enabled = cache
        
        # Initialize crew memory
        if memory:
            self.crew_memory = CrewMemory()
        
        self.execution_results = []
    
    def kickoff(self) -> dict:
        '''
        Start crew execution.
        '''
        if self.verbose:
            print(f'\nCrew starting ({self.process.value} mode)...')
            print(f'Agents: {[a.role for a in self.agents]}')
            print(f'Tasks: {len(self.tasks)}')
            print('-' * 80)
        
        if self.process == ProcessType.SEQUENTIAL:
            return self._execute_sequential()
        elif self.process == ProcessType.HIERARCHICAL:
            return self._execute_hierarchical()
    
    def _execute_sequential(self) -> dict:
        '''Execute tasks sequentially'''
        
        for i, task in enumerate(self.tasks, 1):
            if self.verbose:
                print(f'\nTask {i}/{len(self.tasks)}: {task.description[:50]}...')
            
            # Execute task
            result = task.execute()
            
            # Store result
            self.execution_results.append(result)
            
            # Update crew memory
            if self.memory_enabled:
                self.crew_memory.add_short_term({
                    'task_id': task.task_id,
                    'result': result['result']
                })
            
            if self.verbose:
                print(f'  Status: {result["status"]}')
                print(f'  Time: {result["execution_time"]:.2f}s')
        
        return {
            'results': self.execution_results,
            'total_tasks': len(self.tasks),
            'successful': sum(1 for r in self.execution_results if r['status'] == 'completed'),
            'process': self.process.value
        }
    
    def _execute_hierarchical(self) -> dict:
        '''Execute with manager delegation'''
        
        # In hierarchical mode, manager agent coordinates
        manager = self._get_or_create_manager()
        
        # Manager delegates tasks to specialists
        for task in self.tasks:
            # Manager selects best agent
            selected_agent = manager.select_agent_for_task(task, self.agents)
            task.agent = selected_agent
            
            # Execute
            result = task.execute()
            self.execution_results.append(result)
        
        return {
            'results': self.execution_results,
            'total_tasks': len(self.tasks),
            'successful': sum(1 for r in self.execution_results if r['status'] == 'completed'),
            'process': self.process.value,
            'manager': manager.role
        }
    
    def _get_or_create_manager(self) -> CrewAIAgent:
        '''Get manager agent for hierarchical process'''
        # Check if manager exists
        for agent in self.agents:
            if 'manager' in agent.role.lower():
                return agent
        
        # Create default manager
        return CrewAIAgent(
            role='Project Manager',
            goal='Coordinate team and ensure task completion',
            backstory='Experienced project manager',
            allow_delegation=True
        )

print('CREWAI CREW - COMPLETE ORCHESTRATION')
print('=' * 90)

# Create complete crew
crew = CrewAICrew(
    agents=[research_agent],  # Add more agents as needed
    tasks=[task1, task2, task3],
    process=ProcessType.SEQUENTIAL,
    verbose=True,
    memory=True,
    cache=True
)

# Execute crew
result = crew.kickoff()

print(f'\n{'=' * 90}')
print(f'Crew execution complete!')
print(f'  Total tasks: {result["total_tasks"]}')
print(f'  Successful: {result["successful"]}')
print(f'  Process: {result["process"]}')

print('\n' + '=' * 90)
print('CREW CONFIGURATION OPTIONS:')
print('  - process: sequential (ordered) vs hierarchical (manager delegates)')
print('  - verbose: Enable detailed logging')
print('  - memory: Share context across tasks')
print('  - cache: Reuse results from similar tasks')

### Component 4: Tool - Function Integration

In [ ]:
from pydantic import BaseModel, Field

class CrewAITool:
    '''
    CrewAI Tool for extending agent capabilities.
    
    Features:
    - Name and description
    - Input schema validation
    - Output schema validation
    - Error handling
    - Usage tracking
    - Caching
    '''
    
    def __init__(self,
                 name: str,
                 description: str,
                 func: Callable,
                 args_schema: Optional[BaseModel] = None,
                 cache_results: bool = False):
        
        self.name = name
        self.description = description
        self.func = func
        self.args_schema = args_schema
        self.cache_enabled = cache_results
        
        self.call_count = 0
        self.error_count = 0
        self.cache = {}
    
    def run(self, *args, **kwargs) -> Any:
        '''Execute tool with validation and caching'''
        
        self.call_count += 1
        
        # Check cache
        if self.cache_enabled:
            cache_key = self._get_cache_key(args, kwargs)
            if cache_key in self.cache:
                return self.cache[cache_key]
        
        # Validate args if schema provided
        if self.args_schema:
            try:
                validated_args = self.args_schema(**kwargs)
                kwargs = validated_args.dict()
            except Exception as e:
                self.error_count += 1
                return {'error': f'Invalid arguments: {e}'}
        
        # Execute function
        try:
            result = self.func(*args, **kwargs)
            
            # Cache result
            if self.cache_enabled:
                self.cache[cache_key] = result
            
            return result
            
        except Exception as e:
            self.error_count += 1
            return {'error': str(e)}
    
    def _get_cache_key(self, args, kwargs) -> str:
        '''Generate cache key'''
        key_str = f'{args}:{kwargs}'
        return hashlib.md5(key_str.encode()).hexdigest()[:8]
    
    def get_metrics(self) -> dict:
        '''Get tool usage metrics'''
        return {
            'name': self.name,
            'call_count': self.call_count,
            'error_count': self.error_count,
            'success_rate': (self.call_count - self.error_count) / self.call_count if self.call_count > 0 else 0,
            'cache_size': len(self.cache) if self.cache_enabled else 0
        }

# Define tools
class WebSearchArgs(BaseModel):
    query: str = Field(description='Search query')
    num_results: int = Field(default=5, description='Number of results')

def web_search_func(query: str, num_results: int = 5) -> List[str]:
    '''Mock web search'''
    return [f'Result {i+1} for "{query}"' for i in range(num_results)]

class FileWriteArgs(BaseModel):
    filename: str = Field(description='Output filename')
    content: str = Field(description='Content to write')

def file_write_func(filename: str, content: str) -> str:
    '''Mock file write'''
    return f'Wrote {len(content)} characters to {filename}'

print('CREWAI TOOLS - FUNCTION INTEGRATION')
print('=' * 90)

# Create tools
web_search = CrewAITool(
    name='web_search',
    description='Search the web for information',
    func=web_search_func,
    args_schema=WebSearchArgs,
    cache_results=True
)

file_write = CrewAITool(
    name='file_write',
    description='Write content to a file',
    func=file_write_func,
    args_schema=FileWriteArgs,
    cache_results=False
)

# Use tools
print('\nUsing tools:')
result1 = web_search.run(query='LLM agent frameworks', num_results=3)
print(f'  web_search: {len(result1)} results')

result2 = web_search.run(query='LLM agent frameworks', num_results=3)  # Cache hit
print(f'  web_search (cached): {len(result2)} results')

result3 = file_write.run(filename='report.md', content='Research findings...')
print(f'  file_write: {result3}')

print('\nTool Metrics:')
for tool in [web_search, file_write]:
    metrics = tool.get_metrics()
    print(f'  {metrics["name"]}: {metrics["call_count"]} calls, {metrics["success_rate"]:.0%} success, cache: {metrics["cache_size"]}')

print('\n' + '=' * 90)
print('TOOL DESIGN PATTERNS:')
print('  - Clear name and description')
print('  - Input validation with Pydantic schemas')
print('  - Error handling and reporting')
print('  - Optional caching for expensive operations')
print('  - Usage metrics for monitoring')

### Component 5: Process Types - Sequential vs Hierarchical

In [ ]:
print('PROCESS TYPES - WHEN TO USE EACH')
print('=' * 90)

process_comparison = {
    'Sequential': {
        'Description': 'Tasks execute in order, one after another',
        'Best for': [
            'Pipeline workflows (research → analyze → write)',
            'When task B depends on task A output',
            'Simple, predictable flows',
            'Clear task ordering'
        ],
        'Pros': [
            'Simple to understand and debug',
            'Predictable execution order',
            'Easy to implement',
            'Clear data flow'
        ],
        'Cons': [
            'No parallelism (slower)',
            'One failure blocks entire pipeline',
            'Underutilizes resources'
        ],
        'Example': '''crew = CrewAICrew(
    agents=[researcher, analyst, writer],
    tasks=[research_task, analysis_task, writing_task],
    process=ProcessType.SEQUENTIAL
)'''
    },
    
    'Hierarchical': {
        'Description': 'Manager delegates tasks to specialist agents',
        'Best for': [
            'Large teams (5+ agents)',
            'Complex task assignment logic',
            'Dynamic workload distribution',
            'When expertise matching matters'
        ],
        'Pros': [
            'Intelligent task distribution',
            'Load balancing across agents',
            'Specialist expertise utilized',
            'Scalable to large teams'
        ],
        'Cons': [
            'More complex to set up',
            'Manager can be bottleneck',
            'Requires good manager prompting',
            'Harder to debug'
        ],
        'Example': '''crew = CrewAICrew(
    agents=[manager, specialist1, specialist2, specialist3],
    tasks=tasks,
    process=ProcessType.HIERARCHICAL
)'''
    },
}

for process_name, details in process_comparison.items():
    print(f'\n{process_name} Process')
    print('-' * 80)
    print(f"Description: {details['Description']}\n")
    
    print('Best for:')
    for item in details['Best for']:
        print(f'  • {item}')
    
    print('\nPros:')
    for pro in details['Pros']:
        print(f'  + {pro}')
    
    print('\nCons:')
    for con in details['Cons']:
        print(f'  - {con}')
    
    print(f'\nExample:\n{details["Example"]}')

print('\n' + '=' * 90)
print('DECISION GUIDE:')
print('  - Use SEQUENTIAL for: 3-5 agents, clear dependencies, simple flows')
print('  - Use HIERARCHICAL for: 5+ agents, complex routing, large teams')
print('  - Start with SEQUENTIAL, upgrade to HIERARCHICAL as team grows')

### Component 6: Memory System - Complete Implementation

In [ ]:
class CrewMemorySystem:
    '''
    Complete memory system for CrewAI.
    
    Memory types:
    - Short-term: Recent conversation
    - Long-term: Persistent facts
    - Entity: Track entities across tasks
    - Procedural: Learn from past actions
    '''
    
    def __init__(self):
        self.short_term = deque(maxlen=50)  # Last 50 items
        self.long_term = {}  # Key-value store
        self.entity_memory = defaultdict(dict)  # Entity attributes
        self.procedural = []  # Action patterns
    
    def add_short_term(self, item: dict):
        '''Add to short-term memory'''
        self.short_term.append({
            **item,
            'timestamp': datetime.utcnow().isoformat()
        })
    
    def add_long_term(self, key: str, value: Any, importance: float = 0.5):
        '''Store in long-term memory'''
        self.long_term[key] = {
            'value': value,
            'importance': importance,
            'stored_at': datetime.utcnow().isoformat(),
            'access_count': 0
        }
    
    def get_long_term(self, key: str) -> Optional[Any]:
        '''Retrieve from long-term memory'''
        if key in self.long_term:
            self.long_term[key]['access_count'] += 1
            return self.long_term[key]['value']
        return None
    
    def track_entity(self, entity_name: str, attributes: dict):
        '''Track entity across tasks'''
        self.entity_memory[entity_name].update(attributes)
        self.entity_memory[entity_name]['last_updated'] = datetime.utcnow().isoformat()
    
    def get_entity(self, entity_name: str) -> dict:
        '''Get entity information'''
        return dict(self.entity_memory.get(entity_name, {}))
    
    def learn_procedure(self, action: str, outcome: str, success: bool):
        '''Learn from action outcomes'''
        self.procedural.append({
            'action': action,
            'outcome': outcome,
            'success': success,
            'timestamp': datetime.utcnow().isoformat()
        })
    
    def get_relevant_memories(self, query: str, top_k: int = 5) -> dict:
        '''Get relevant memories for current task'''
        # In production: use embedding similarity
        # For demo: return recent items
        return {
            'short_term': list(self.short_term)[-top_k:],
            'long_term_count': len(self.long_term),
            'entities_tracked': len(self.entity_memory),
            'procedures_learned': len(self.procedural)
        }
    
    def summarize(self) -> dict:
        '''Get memory statistics'''
        return {
            'short_term_items': len(self.short_term),
            'long_term_facts': len(self.long_term),
            'entities_tracked': len(self.entity_memory),
            'procedures_learned': len(self.procedural),
        }

print('CREWAI MEMORY SYSTEM')
print('=' * 90)

memory = CrewMemorySystem()

# Demonstrate memory usage
print('\nAdding memories...')

# Short-term
memory.add_short_term({'task': 'Research LLMs', 'result': 'Found 5 papers'})
memory.add_short_term({'task': 'Analyze results', 'result': 'Key insight: Agents improve quality'})

# Long-term
memory.add_long_term('company_name', 'TechCorp', importance=0.9)
memory.add_long_term('project_deadline', '2025-12-31', importance=0.8)

# Entity tracking
memory.track_entity('GPT-4', {'type': 'LLM', 'vendor': 'OpenAI', 'cost_per_1k': 0.03})
memory.track_entity('Claude', {'type': 'LLM', 'vendor': 'Anthropic', 'cost_per_1k': 0.024})

# Procedural learning
memory.learn_procedure('web_search', 'Found 10 results', success=True)
memory.learn_procedure('api_call', 'Timeout error', success=False)

print('\nMemory summary:')
summary = memory.summarize()
for key, value in summary.items():
    print(f'  {key}: {value}')

print('\nEntity example:')
print(f'  GPT-4: {memory.get_entity("GPT-4")}')

print('\n' + '=' * 90)
print('MEMORY BENEFITS:')
print('  - Short-term: Conversation context')
print('  - Long-term: Important facts preserved')
print('  - Entity: Track objects across tasks')
print('  - Procedural: Learn from experience')
print('  - Shared across all crew agents')

### Component 7: Output Parser - Structured Output Validation

In [ ]:
class CrewAIOutputParser:
    '''
    Parse and validate crew outputs.
    
    Features:
    - Schema validation
    - Multiple format support (JSON, Markdown, custom)
    - Extraction from LLM responses
    - Retry on parse failure
    '''
    
    def __init__(self, output_schema: Optional[BaseModel] = None, format_type: str = 'json'):
        self.output_schema = output_schema
        self.format_type = format_type
    
    def parse(self, raw_output: str, max_retries: int = 2) -> Any:
        '''Parse and validate output'''
        
        for attempt in range(max_retries):
            try:
                if self.format_type == 'json':
                    return self._parse_json(raw_output)
                elif self.format_type == 'markdown':
                    return self._parse_markdown(raw_output)
                else:
                    return raw_output
                    
            except Exception as e:
                if attempt == max_retries - 1:
                    raise ValueError(f'Failed to parse output: {e}')
    
    def _parse_json(self, raw_output: str) -> Any:
        '''Extract and validate JSON'''
        # Extract JSON from markdown code blocks
        if '```json' in raw_output:
            json_str = raw_output.split('```json')[1].split('```')[0].strip()
        elif '```' in raw_output:
            json_str = raw_output.split('```')[1].split('```')[0].strip()
        else:
            json_str = raw_output.strip()
        
        # Parse JSON
        data = json.loads(json_str)
        
        # Validate against schema
        if self.output_schema:
            validated = self.output_schema(**data)
            return validated
        
        return data
    
    def _parse_markdown(self, raw_output: str) -> dict:
        '''Parse markdown output'''
        # Extract sections
        sections = {}
        current_section = 'content'
        current_content = []
        
        for line in raw_output.split('\n'):
            if line.startswith('##'):
                # Save previous section
                if current_content:
                    sections[current_section] = '\n'.join(current_content)
                
                # Start new section
                current_section = line.replace('#', '').strip()
                current_content = []
            else:
                current_content.append(line)
        
        # Save last section
        if current_content:
            sections[current_section] = '\n'.join(current_content)
        
        return sections

class ReportOutput(BaseModel):
    '''Example output schema for report'''
    title: str
    executive_summary: str
    key_findings: List[str]
    recommendations: List[str]
    confidence_score: float = Field(ge=0.0, le=1.0)

print('OUTPUT PARSER DEMONSTRATION')
print('=' * 90)

# Create parser with schema
parser = CrewAIOutputParser(
    output_schema=ReportOutput,
    format_type='json'
)

# Mock LLM output
llm_output = '''```json
{
  "title": "LLM Agent Analysis",
  "executive_summary": "Analysis of current LLM agent frameworks.",
  "key_findings": [
    "AutoGen excels at code generation",
    "CrewAI best for role-based delegation",
    "LangGraph ideal for complex workflows"
  ],
  "recommendations": [
    "Use AutoGen for coding tasks",
    "Use CrewAI for content creation"
  ],
  "confidence_score": 0.85
}
```'''

print('\nParsing LLM output...')
try:
    parsed = parser.parse(llm_output)
    print(f'✓ Successfully parsed and validated')
    print(f'  Title: {parsed.title}')
    print(f'  Key findings: {len(parsed.key_findings)}')
    print(f'  Confidence: {parsed.confidence_score}')
except Exception as e:
    print(f'✗ Parse failed: {e}')

print('\n' + '=' * 90)
print('OUTPUT PARSER BENEFITS:')
print('  - Ensures output matches expected format')
print('  - Automatic extraction from markdown blocks')
print('  - Schema validation with Pydantic')
print('  - Retry on parse failure')
print('  - Multiple format support')

### Component 8: Flow - Complex Multi-Step Workflows

In [ ]:
class CrewAIFlow:
    '''
    CrewAI Flow for complex multi-crew workflows.
    
    Features:
    - Orchestrate multiple crews
    - Conditional routing between crews
    - State management across crews
    - Error handling and recovery
    '''
    
    def __init__(self, name: str):
        self.name = name
        self.crews = {}
        self.flow_state = {}
        self.flow_history = []
    
    def add_crew(self, crew_name: str, crew: CrewAICrew):
        '''Add crew to flow'''
        self.crews[crew_name] = crew
    
    def execute_flow(self, initial_input: dict, flow_steps: List[dict]) -> dict:
        '''
        Execute complex flow.
        
        flow_steps format:
        [
            {'crew': 'research_crew', 'condition': lambda state: True},
            {'crew': 'analysis_crew', 'condition': lambda state: state['quality'] > 0.7},
            ...
        ]
        '''
        
        self.flow_state = initial_input
        
        for step_num, step in enumerate(flow_steps, 1):
            crew_name = step['crew']
            condition = step.get('condition', lambda s: True)
            
            # Check condition
            if not condition(self.flow_state):
                print(f'Step {step_num}: Skipping {crew_name} (condition not met)')
                continue
            
            print(f'\nStep {step_num}: Executing {crew_name}...')
            
            # Execute crew
            if crew_name in self.crews:
                crew_result = self.crews[crew_name].kickoff()
                
                # Update flow state
                self.flow_state[crew_name] = crew_result
                
                # Store in history
                self.flow_history.append({
                    'step': step_num,
                    'crew': crew_name,
                    'result': crew_result
                })
            else:
                print(f'  Warning: Crew {crew_name} not found')
        
        return {
            'flow_name': self.name,
            'steps_executed': len(self.flow_history),
            'final_state': self.flow_state,
            'history': self.flow_history
        }

print('CREWAI FLOW - COMPLEX WORKFLOWS')
print('=' * 90)

# Example: Multi-stage content creation flow
flow = CrewAIFlow('content_creation_pipeline')

# Create crews for different stages
research_crew = CrewAICrew(
    agents=[research_agent],
    tasks=[task1],
    process=ProcessType.SEQUENTIAL
)

flow.add_crew('research', research_crew)
# flow.add_crew('analysis', analysis_crew)  # Would add more
# flow.add_crew('writing', writing_crew)

# Define flow steps
flow_steps = [
    {'crew': 'research', 'condition': lambda s: True},
    # {'crew': 'analysis', 'condition': lambda s: s.get('research_quality', 0) > 0.7},
    # {'crew': 'writing', 'condition': lambda s: s.get('analysis_complete', False)},
]

print('\nExecuting flow...')
result = flow.execute_flow(
    initial_input={'topic': 'LLM Agents'},
    flow_steps=flow_steps
)

print(f'\nFlow complete: {result["steps_executed"]} steps executed')

print('\n' + '=' * 90)
print('FLOW BENEFITS:')
print('  - Orchestrate multiple crews')
print('  - Conditional routing between stages')
print('  - State management across workflow')
print('  - Error recovery at crew level')
print('  - Complex multi-stage pipelines')

### Component 9: Config - YAML-Based Setup

Define crews and agents declaratively.

In [ ]:
import yaml

class CrewAIConfig:
    '''
    Load CrewAI configuration from YAML.
    
    Enables:
    - Declarative crew definition
    - Version control of configs
    - Easy modifications without code changes
    - Environment-specific configs
    '''
    
    def __init__(self, config_file: str = None):
        self.config = {}
        if config_file:
            self.load_from_file(config_file)
    
    def load_from_file(self, file_path: str):
        '''Load configuration from YAML file'''
        with open(file_path, 'r') as f:
            self.config = yaml.safe_load(f)
    
    def load_from_dict(self, config_dict: dict):
        '''Load from dictionary'''
        self.config = config_dict
    
    def build_crew(self) -> CrewAICrew:
        '''Build crew from configuration'''
        
        # Create agents from config
        agents = []
        for agent_config in self.config.get('agents', []):
            agent = CrewAIAgent(
                role=agent_config['role'],
                goal=agent_config['goal'],
                backstory=agent_config['backstory'],
                tools=agent_config.get('tools', []),
                verbose=agent_config.get('verbose', True),
                allow_delegation=agent_config.get('allow_delegation', False),
                memory=agent_config.get('memory', True)
            )
            agents.append(agent)
        
        # Create tasks from config
        tasks = []
        agent_map = {a.role: a for a in agents}
        
        for task_config in self.config.get('tasks', []):
            # Find agent by role
            agent = agent_map.get(task_config.get('agent_role'))
            
            task = CrewAITask(
                description=task_config['description'],
                expected_output=task_config['expected_output'],
                agent=agent,
                async_execution=task_config.get('async', False)
            )
            tasks.append(task)
        
        # Create crew
        crew_config = self.config.get('crew', {})
        process_str = crew_config.get('process', 'sequential')
        process = ProcessType.SEQUENTIAL if process_str == 'sequential' else ProcessType.HIERARCHICAL
        
        crew = CrewAICrew(
            agents=agents,
            tasks=tasks,
            process=process,
            verbose=crew_config.get('verbose', True),
            memory=crew_config.get('memory', True)
        )
        
        return crew

print('CREWAI CONFIG - YAML SETUP')
print('=' * 90)

# Example configuration
example_config = {
    'agents': [
        {
            'role': 'Research Specialist',
            'goal': 'Find comprehensive information',
            'backstory': 'Expert researcher with access to multiple sources',
            'tools': ['web_search', 'arxiv_search'],
            'allow_delegation': False,
            'memory': True
        },
        {
            'role': 'Technical Writer',
            'goal': 'Create clear technical documentation',
            'backstory': 'Technical writer with 8 years experience',
            'tools': ['markdown_formatter'],
            'allow_delegation': False,
            'memory': True
        }
    ],
    'tasks': [
        {
            'description': 'Research latest RAG techniques',
            'expected_output': 'Summary of 5 techniques with sources',
            'agent_role': 'Research Specialist',
            'async': False
        },
        {
            'description': 'Write technical blog post about research',
            'expected_output': '1500-word blog post',
            'agent_role': 'Technical Writer',
            'async': False
        }
    ],
    'crew': {
        'process': 'sequential',
        'verbose': True,
        'memory': True
    }
}

print('\nExample Configuration:')
print(yaml.dump(example_config, default_flow_style=False, indent=2)[:500] + '...')

# Build crew from config
config_manager = CrewAIConfig()
config_manager.load_from_dict(example_config)

try:
    configured_crew = config_manager.build_crew()
    print(f'\n✓ Crew built from config:')
    print(f'  Agents: {len(configured_crew.agents)}')
    print(f'  Tasks: {len(configured_crew.tasks)}')
    print(f'  Process: {configured_crew.process.value}')
except Exception as e:
    print(f'\n✗ Error building crew: {e}')

print('\n' + '=' * 90)
print('CONFIG BENEFITS:')
print('  - Version control configurations')
print('  - No code changes for crew modifications')
print('  - Environment-specific configs (dev, staging, prod)')
print('  - Easy experimentation with different setups')
print('  - Shareable configurations across team')